# Reinforcement Learning mit Python - Running Kings Schach KI
## Aufgabenstellung

Erstellen  Sie  Modelle,  welche  das  Spiel  Racing  Kings erlernen.  Am  Ende  sollte  ein  Spiel  gegen  den erstellten  Algorithmus  möglich  sein.  Testen  Sie  als  Gegner  einenAlgorithmus,  welcher  Zufallszüge ausführt,  und  bewerten  Sie  ihren  Algorithmus. Nutzen  Sie  die  chess-Bibliothek  (https://python-chess.readthedocs.io/en/latest/)

Bonus: Vergleichen Sie einen modellfreien und einen modellbasierten Ansatz.

Diese Anleitung wurde verwendet um zügig mit der verwendung der chess bibliothek starten zu können.
https://jupyter.brynmawr.edu/services/public/dblank/CS371%20Cognitive%20Science/2016-Fall/Programming%20a%20Chess%20Player.ipynb

# Install Dependencies

In [3]:
!pip install numpy
!pip install tensorflow==2.3.0
!pip install keras
!pip install keras-rl2
!pip install chess
!pip install pydot
!pip install gym
!pip install stable_baselines3

You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/armin/CloudStation/06_hochschule/02_MAPR/04_vorlesungen/06_einführung_in_die_ki_mit_python/code/env/bin/python3 -m pip install --upgrade pip' command.


# Create chess environment

In [4]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

In [5]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

In [18]:
import chess.variant
import random
import time
from IPython.display import display, HTML, clear_output
import numpy as np
import gym
from gym.spaces import Discrete, Box, Dict

N_DISCRETE_ACTIONS = 4096

# from Learning_Chess pdf
class RacingKingsEnvironment(gym.Env):
    def __init__(self):
        super(RacingKingsEnvironment, self).__init__()
        self.board = chess.variant.RacingKingsBoard()
        self.reward = 0
        self.action_space = Discrete(N_DISCRETE_ACTIONS)
        self.observation_space = Box(low=0, high=1, shape=(14, 8, 8), dtype=np.uint8)
    def board_square_to_index(self, name):
        return (int(name[1])-1) * 8 + (ord(name[0])-97) 
    def action_index_to_uci(self, index):
        index_from = index//64
        index_to = index%64
        name = chr(index_from%8 + 97) + str(index_from//8 +1) + chr(index_to%8 + 97) + str(index_to//8 + 1)
        return name
    def action_uci_to_index(self, uci):
        index_from = (int(uci[1])-1) * 8 + (ord(uci[0])-97) 
        index_to = (int(uci[3])-1) * 8 + (ord(uci[2])-97) 
        return index_from*64 + index_to
    @property
    def actions(self):
        moves = list(self.board.legal_moves)
        moves_string = []
        for move in moves:
            moves_string.append(move.uci())
        boardActions = np.zeros(N_DISCRETE_ACTIONS, dtype=np.int8)
        for move in moves_string:
            boardActions[self.board_square_to_index(move[0:2])*64 + self.board_square_to_index(move[2:4])] = 1
        return boardActions
    @property
    def states(self):
        boardState = np.zeros((14, 8, 8), dtype=np.int8)
        for piece in chess.PIECE_TYPES:
            for square in self.board.pieces(piece, chess.WHITE):
                idNum = square//8
                idAlph = square%8
                boardState[piece - 1][7 - idNum][idAlph] = 1
            for square in self.board.pieces(piece, chess.BLACK):
                idNum = square//8
                idAlph = square%8
                boardState[piece + 5][7 - idNum][idAlph] = 1        
        
            temp = self.board.turn
            self.board.turn = chess.WHITE
            for move in list(self.board.legal_moves):
                square = self.board_square_to_index(move.uci())
                idNum = square//8
                idAlph = square%8
                boardState[12][7 - idNum][idAlph] = 1
            self.board.turn = chess.BLACK
            for move in list(self.board.legal_moves):
                square = self.board_square_to_index(move.uci())
                idNum = square//8
                idAlph = square%8
                boardState[13][7 - idNum][idAlph] = 1
            self.board.turn = temp
        return boardState
    
    def step(self, action):
        done = False
        step_reward = 0
        info = {}
        
# check if it is not your turn -> then make a random move
# when uncommenting this code the ai will play moves for both players
        if self.board.turn == chess.WHITE:
            try:
                self.board.push(random.choice(list(self.board.legal_moves)))
                step_reward += 0
                info = {"msg":"White Did a valid move"}
            except:
                info = {"msg":"Passed an already finished board"} 
                done = True

        if not self.board.is_game_over(claim_draw=True):
            if action is not None:
                try:
                    self.board.push_uci(self.action_index_to_uci(action))
                    step_reward += 0.1
                    info = {"msg":"Did a valid move"}
                except:
                    step_reward -= 1
                    info = {"msg":"Action is not a valid move"} 
                    done = True
                if self.board.is_variant_end():
                    if who(not self.board.turn) == "Black":
                        step_reward += 1
                        info = {"msg":"AI won the game (being black)!"} 
                        done = True
                    else:
                        info = {"msg":"Random Player won!"} 
                        done = True
                    info = {"msg":"racing kings: " + who(not self.board.turn) + " wins!"}
        else:
            step_reward -=1
            done = True
            info = {"msg":"game over"}

        self.reward += step_reward
        
        return self.states, step_reward, done, info
    def reset(self):
        # reset the board
        self.board.reset()
        # play random amount of actions
        for i in range((random.randint(0, 50)*2)):
            try:
                move = random.choice(list(self.board.legal_moves))
                self.board.push(move)
            except:
                self.board.reset()
                
        self.reward = 0.0
        return self.step(None)[0]  # reward, done, info can't be included
    def render(self, mode="human", pause=0.5):
        name = who(self.board.turn)
        use_svg = (mode == "human")
        board_stop = display_board(self.board, use_svg)
        html = "<b>Move %s %s:</b><br/>%s" % (
                    len(self.board.move_stack), name, board_stop)
        if mode is not None:
                if mode == "human":
                    clear_output(wait=True)
                display(HTML(html))
                if mode == "human":
                    time.sleep(pause)
    def close (self):
        print("closing")

# Test Chess environment randomly

In [21]:
episodes = 10
env = RacingKingsEnvironment()
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render(mode=None)
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} Info:{}'.format(episode, score, info))
env.close()

Episode:1 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:2 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:3 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:4 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:5 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:6 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:7 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:8 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:9 Score:-1 Info:{'msg': 'Action is not a valid move'}
Episode:10 Score:-1 Info:{'msg': 'Action is not a valid move'}
closing


# Training Model with stable baselines

In [22]:
from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy 
from stable_baselines3.common.callbacks import EvalCallback 
import os

In [23]:
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = RacingKingsEnvironment()
env = DummyVecEnv([lambda: env])
model = DQN('CNNPolicy', env, verbose=1, tensorboard_log=log_path)

In [26]:
best_save_path = os.path.join('Training', 'SavedModels', 'Best_Racing_Kings')
eval_callback = EvalCallback(env, eval_freq=1000, best_model_save_path = best_save_path, verbose=1)

In [ ]:
model.learn(total_timesteps=100000, callback=eval_callback)

Logging to Training/Logs/PPO_35
Eval num_timesteps=187, episode_reward=-1.00 +/- 0.00
Episode length: 1.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 1        |
|    mean_reward     | -1       |
| time/              |          |
|    total timesteps | 187      |
---------------------------------
Eval num_timesteps=1187, episode_reward=-1.00 +/- 0.00
Episode length: 1.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 1        |
|    mean_reward     | -1       |
| time/              |          |
|    total timesteps | 1187     |
---------------------------------
-----------------------------
| time/              |      |
|    fps             | 17   |
|    iterations      | 1    |
|    time_elapsed    | 114  |
|    total_timesteps | 2048 |
-----------------------------
Eval num_timesteps=2187, episode_reward=-1.00 +/- 0.00
Episode length: 1.00 +/- 0.00
----------------------------

Saving the model

In [12]:
Model_Path = os.path.join('Training', 'SavedModels', 'PPO_35_Racing_Kings_100k_win+step_reward_random_start_after_reset_random_opponent')

In [ ]:
model.save(Model_Path)

In [13]:
del model

In [14]:
model = PPO.load(Model_Path, env=env)

# Evaluate the Model

## Testing the Model

In [15]:
episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render(mode=None)
        action = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} Info:{}'.format(episode, score, info))

Episode:1 Score:[0.] Info:[{'msg': 'Action is not a valid move', 'terminal_observation': array([[[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 

Episode:5 Score:[0.] Info:[{'msg': 'Action is not a valid move', 'terminal_observation': array([[[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 

Episode:9 Score:[0.] Info:[{'msg': 'Action is not a valid move', 'terminal_observation': array([[[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 

In [16]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(0.0, 0.0)